In [1]:
MAX_ROUNDS = 650
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.05

In [2]:
DEBUG = False
if DEBUG:
    NROWS = 5000
else:
    NROWS = None

In [3]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from numba import jit

In [4]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [5]:
# Read data
train_df = pd.read_csv('data/train.csv', na_values="-1", nrows=NROWS) # .iloc[0:200,:]
test_df = pd.read_csv('data/test.csv', na_values="-1", nrows=NROWS)

In [6]:
# Process data
id_test = test_df['id'].values
id_train = train_df['id'].values

train_df = train_df.fillna(999)
test_df = test_df.fillna(999)

col_to_drop = train_df.columns[train_df.columns.str.startswith('ps_calc_')]
train_df = train_df.drop(col_to_drop, axis=1)  
test_df = test_df.drop(col_to_drop, axis=1)  

for c in train_df.select_dtypes(include=['float64']).columns:
    train_df[c]=train_df[c].astype(np.float32)
    test_df[c]=test_df[c].astype(np.float32)
for c in train_df.select_dtypes(include=['int64']).columns[2:]:
    train_df[c]=train_df[c].astype(np.int8)
    test_df[c]=test_df[c].astype(np.int8)
    
y = train_df['target']
X = train_df.drop(['target', 'id'], axis=1)
y_valid_pred = 0*y
X_test = test_df.drop(['id'], axis=1)
y_test_pred = 0

In [7]:
y_valid_pred

0         0
1         0
2         0
3         0
4         0
         ..
595207    0
595208    0
595209    0
595210    0
595211    0
Name: target, Length: 595212, dtype: int64

In [8]:
y_test_pred

0

In [9]:
# Set up folds
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)

In [10]:
# Set up classifier
model = CatBoostClassifier(
    learning_rate=LEARNING_RATE, 
    depth=6, 
    l2_leaf_reg = 14, 
    iterations = MAX_ROUNDS,
#    verbose = True,
    loss_function='Logloss'
)

In [ ]:
# Run CV

for i, (train_index, valid_index) in enumerate(kf.split(train_df)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
    print( "\nFold ", i)
    
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        fit_model = model.fit( X_train, y_train, 
                               eval_set=(X_valid, y_valid),
                               use_best_model=True
                             )
        print( "  N trees = ", model.tree_count_ )
    else:
        fit_model = model.fit( X_train, y_train )
        
    # Generate validation predictions of target=1 for this fold
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[valid_index] = pred
    
    # Accumulate test set predictions
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
y_test_pred /= K  # Average test set predictions

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
0:	learn: 0.6201384	total: 386ms	remaining: 4m 10s
1:	learn: 0.5568259	total: 632ms	remaining: 3m 24s
2:	learn: 0.5020721	total: 967ms	remaining: 3m 28s
3:	learn: 0.4549206	total: 1.39s	remaining: 3m 45s
4:	learn: 0.4148491	total: 1.64s	remaining: 3m 31s
5:	learn: 0.3801773	total: 1.89s	remaining: 3m 22s
6:	learn: 0.3505809	total: 2.12s	remaining: 3m 15s
7:	learn: 0.3247216	total: 2.36s	remaining: 3m 9s
8:	learn: 0.3026544	total: 2.6s	remaining: 3m 5s
9:	learn: 0.2836941	total: 2.84s	remaining: 3m 1s
10:	learn: 0.2674488	total: 3.07s	remaining: 2m 58s
11:	learn: 0.2532570	total: 3.3s	remaining: 2m 55s
12:	learn: 0.2408090	total: 3.54s	remaining: 2m 53s
13:	learn: 0.2302570	total: 3.79s	remaining: 2m 51s
14:	learn: 0.2209849	total: 4.03s	remaining: 2m 50s
15:	learn: 0.2130008	total: 4.26s	remaining: 2m 48s
16:	learn: 0.2059982	total: 4.51s	remaining: 2m 47s
17:	learn: 0.1998278	total: 4.76s	remaining: 2m 47s
18:	learn: 0.1945168	total: 5s	remaining: 2m 46s
19:	learn: 0.1897779	

158:	learn: 0.1513084	total: 40.3s	remaining: 2m 4s
159:	learn: 0.1512936	total: 40.5s	remaining: 2m 4s
160:	learn: 0.1512849	total: 40.7s	remaining: 2m 3s
161:	learn: 0.1512788	total: 41s	remaining: 2m 3s
162:	learn: 0.1512692	total: 41.2s	remaining: 2m 3s
163:	learn: 0.1512628	total: 41.5s	remaining: 2m 2s
164:	learn: 0.1512556	total: 41.7s	remaining: 2m 2s
165:	learn: 0.1512498	total: 42s	remaining: 2m 2s
166:	learn: 0.1512392	total: 42.2s	remaining: 2m 2s
167:	learn: 0.1512257	total: 42.5s	remaining: 2m 1s
168:	learn: 0.1512165	total: 42.7s	remaining: 2m 1s
169:	learn: 0.1512046	total: 43s	remaining: 2m 1s
170:	learn: 0.1511984	total: 43.2s	remaining: 2m 1s
171:	learn: 0.1511899	total: 43.4s	remaining: 2m
172:	learn: 0.1511804	total: 43.7s	remaining: 2m
173:	learn: 0.1511706	total: 43.9s	remaining: 2m
174:	learn: 0.1511599	total: 44.2s	remaining: 1m 59s
175:	learn: 0.1511537	total: 44.4s	remaining: 1m 59s
176:	learn: 0.1511477	total: 44.7s	remaining: 1m 59s
177:	learn: 0.1511416	to

313:	learn: 0.1500661	total: 1m 18s	remaining: 1m 23s
314:	learn: 0.1500560	total: 1m 18s	remaining: 1m 23s
315:	learn: 0.1500476	total: 1m 18s	remaining: 1m 23s
316:	learn: 0.1500406	total: 1m 19s	remaining: 1m 23s
317:	learn: 0.1500302	total: 1m 19s	remaining: 1m 22s
318:	learn: 0.1500225	total: 1m 19s	remaining: 1m 22s
319:	learn: 0.1500116	total: 1m 19s	remaining: 1m 22s
320:	learn: 0.1500035	total: 1m 20s	remaining: 1m 22s
321:	learn: 0.1499945	total: 1m 20s	remaining: 1m 21s
322:	learn: 0.1499845	total: 1m 20s	remaining: 1m 21s
323:	learn: 0.1499789	total: 1m 20s	remaining: 1m 21s
324:	learn: 0.1499710	total: 1m 21s	remaining: 1m 21s
325:	learn: 0.1499658	total: 1m 21s	remaining: 1m 20s
326:	learn: 0.1499582	total: 1m 21s	remaining: 1m 20s
327:	learn: 0.1499525	total: 1m 21s	remaining: 1m 20s
328:	learn: 0.1499473	total: 1m 22s	remaining: 1m 20s
329:	learn: 0.1499410	total: 1m 22s	remaining: 1m 19s
330:	learn: 0.1499330	total: 1m 22s	remaining: 1m 19s
331:	learn: 0.1499236	total:

468:	learn: 0.1491713	total: 1m 56s	remaining: 45s
469:	learn: 0.1491636	total: 1m 56s	remaining: 44.8s
470:	learn: 0.1491617	total: 1m 57s	remaining: 44.5s
471:	learn: 0.1491570	total: 1m 57s	remaining: 44.3s
472:	learn: 0.1491490	total: 1m 57s	remaining: 44s
473:	learn: 0.1491490	total: 1m 57s	remaining: 43.8s
474:	learn: 0.1491450	total: 1m 58s	remaining: 43.5s
475:	learn: 0.1491365	total: 1m 58s	remaining: 43.3s
476:	learn: 0.1491295	total: 1m 58s	remaining: 43s
477:	learn: 0.1491295	total: 1m 58s	remaining: 42.8s
478:	learn: 0.1491191	total: 1m 59s	remaining: 42.5s
479:	learn: 0.1491162	total: 1m 59s	remaining: 42.3s
480:	learn: 0.1491161	total: 1m 59s	remaining: 42s
481:	learn: 0.1491062	total: 1m 59s	remaining: 41.8s
482:	learn: 0.1491010	total: 2m	remaining: 41.5s
483:	learn: 0.1490943	total: 2m	remaining: 41.3s
484:	learn: 0.1490858	total: 2m	remaining: 41.1s
485:	learn: 0.1490789	total: 2m 1s	remaining: 40.8s
486:	learn: 0.1490740	total: 2m 1s	remaining: 40.6s
487:	learn: 0.1

624:	learn: 0.1482843	total: 2m 39s	remaining: 6.39s
625:	learn: 0.1482812	total: 2m 40s	remaining: 6.13s
626:	learn: 0.1482812	total: 2m 40s	remaining: 5.88s
627:	learn: 0.1482768	total: 2m 40s	remaining: 5.62s
628:	learn: 0.1482739	total: 2m 40s	remaining: 5.37s
629:	learn: 0.1482673	total: 2m 40s	remaining: 5.11s
630:	learn: 0.1482574	total: 2m 41s	remaining: 4.86s
631:	learn: 0.1482527	total: 2m 41s	remaining: 4.6s
632:	learn: 0.1482491	total: 2m 41s	remaining: 4.34s
633:	learn: 0.1482420	total: 2m 41s	remaining: 4.09s
634:	learn: 0.1482353	total: 2m 42s	remaining: 3.83s
635:	learn: 0.1482300	total: 2m 42s	remaining: 3.58s
636:	learn: 0.1482215	total: 2m 42s	remaining: 3.32s
637:	learn: 0.1482165	total: 2m 42s	remaining: 3.06s
638:	learn: 0.1482100	total: 2m 43s	remaining: 2.81s
639:	learn: 0.1482060	total: 2m 43s	remaining: 2.55s
640:	learn: 0.1482035	total: 2m 43s	remaining: 2.3s
641:	learn: 0.1481952	total: 2m 43s	remaining: 2.04s
642:	learn: 0.1481879	total: 2m 44s	remaining: 1

...

543:	learn: 0.1491984	total: 2m 53s	remaining: 33.7s
544:	learn: 0.1491888	total: 2m 53s	remaining: 33.4s
545:	learn: 0.1491836	total: 2m 53s	remaining: 33.1s
546:	learn: 0.1491785	total: 2m 53s	remaining: 32.7s
547:	learn: 0.1491732	total: 2m 54s	remaining: 32.4s
548:	learn: 0.1491700	total: 2m 54s	remaining: 32.1s
549:	learn: 0.1491632	total: 2m 54s	remaining: 31.7s
550:	learn: 0.1491591	total: 2m 54s	remaining: 31.4s
551:	learn: 0.1491528	total: 2m 55s	remaining: 31.1s
552:	learn: 0.1491460	total: 2m 55s	remaining: 30.8s
553:	learn: 0.1491412	total: 2m 55s	remaining: 30.4s
554:	learn: 0.1491380	total: 2m 55s	remaining: 30.1s
555:	learn: 0.1491339	total: 2m 56s	remaining: 29.8s
556:	learn: 0.1491275	total: 2m 56s	remaining: 29.5s
557:	learn: 0.1491208	total: 2m 56s	remaining: 29.1s
558:	learn: 0.1491149	total: 2m 56s	remaining: 28.8s
559:	learn: 0.1491149	total: 2m 57s	remaining: 28.5s
560:	learn: 0.1491077	total: 2m 57s	remaining: 28.1s
561:	learn: 0.1491062	total: 2m 57s	remaining:

50:	learn: 0.1541487	total: 12.6s	remaining: 2m 28s
51:	learn: 0.1539849	total: 12.9s	remaining: 2m 28s
52:	learn: 0.1538395	total: 13.2s	remaining: 2m 28s
53:	learn: 0.1537242	total: 13.4s	remaining: 2m 28s
54:	learn: 0.1536044	total: 13.7s	remaining: 2m 28s
55:	learn: 0.1535028	total: 13.9s	remaining: 2m 27s
56:	learn: 0.1534019	total: 14.2s	remaining: 2m 27s
57:	learn: 0.1533283	total: 14.4s	remaining: 2m 27s
58:	learn: 0.1532463	total: 14.7s	remaining: 2m 27s
59:	learn: 0.1531739	total: 14.9s	remaining: 2m 26s
60:	learn: 0.1530999	total: 15.2s	remaining: 2m 26s
61:	learn: 0.1530292	total: 15.4s	remaining: 2m 26s
62:	learn: 0.1529606	total: 15.7s	remaining: 2m 26s
63:	learn: 0.1528956	total: 16s	remaining: 2m 26s
64:	learn: 0.1528362	total: 16.2s	remaining: 2m 26s
65:	learn: 0.1527856	total: 16.5s	remaining: 2m 25s
66:	learn: 0.1527344	total: 16.7s	remaining: 2m 25s
67:	learn: 0.1526686	total: 17s	remaining: 2m 25s
68:	learn: 0.1526256	total: 17.3s	remaining: 2m 25s
69:	learn: 0.152

208:	learn: 0.1506431	total: 52.7s	remaining: 1m 51s
209:	learn: 0.1506344	total: 53s	remaining: 1m 50s
210:	learn: 0.1506296	total: 53.2s	remaining: 1m 50s
211:	learn: 0.1506232	total: 53.4s	remaining: 1m 50s
212:	learn: 0.1506148	total: 53.7s	remaining: 1m 50s
213:	learn: 0.1506079	total: 53.9s	remaining: 1m 49s
214:	learn: 0.1506019	total: 54.2s	remaining: 1m 49s
215:	learn: 0.1505975	total: 54.4s	remaining: 1m 49s
216:	learn: 0.1505890	total: 54.7s	remaining: 1m 49s
217:	learn: 0.1505824	total: 54.9s	remaining: 1m 48s
218:	learn: 0.1505760	total: 55.2s	remaining: 1m 48s
219:	learn: 0.1505655	total: 55.4s	remaining: 1m 48s
220:	learn: 0.1505581	total: 55.6s	remaining: 1m 48s
221:	learn: 0.1505515	total: 55.9s	remaining: 1m 47s
222:	learn: 0.1505437	total: 56.1s	remaining: 1m 47s
223:	learn: 0.1505362	total: 56.4s	remaining: 1m 47s
224:	learn: 0.1505358	total: 56.6s	remaining: 1m 46s
225:	learn: 0.1505289	total: 56.8s	remaining: 1m 46s
226:	learn: 0.1505235	total: 57.1s	remaining: 1m

362:	learn: 0.1496223	total: 1m 30s	remaining: 1m 11s
363:	learn: 0.1496134	total: 1m 30s	remaining: 1m 11s
364:	learn: 0.1496087	total: 1m 31s	remaining: 1m 11s
365:	learn: 0.1496024	total: 1m 31s	remaining: 1m 10s
366:	learn: 0.1495959	total: 1m 31s	remaining: 1m 10s
367:	learn: 0.1495850	total: 1m 31s	remaining: 1m 10s
368:	learn: 0.1495839	total: 1m 32s	remaining: 1m 10s
369:	learn: 0.1495713	total: 1m 32s	remaining: 1m 9s
370:	learn: 0.1495665	total: 1m 32s	remaining: 1m 9s
371:	learn: 0.1495610	total: 1m 32s	remaining: 1m 9s
372:	learn: 0.1495539	total: 1m 33s	remaining: 1m 9s
373:	learn: 0.1495481	total: 1m 33s	remaining: 1m 8s
374:	learn: 0.1495413	total: 1m 33s	remaining: 1m 8s
375:	learn: 0.1495339	total: 1m 33s	remaining: 1m 8s
376:	learn: 0.1495267	total: 1m 34s	remaining: 1m 8s
377:	learn: 0.1495209	total: 1m 34s	remaining: 1m 7s
378:	learn: 0.1495101	total: 1m 34s	remaining: 1m 7s
379:	learn: 0.1495003	total: 1m 34s	remaining: 1m 7s
380:	learn: 0.1494956	total: 1m 35s	rem

518:	learn: 0.1486835	total: 2m 9s	remaining: 32.6s
519:	learn: 0.1486725	total: 2m 9s	remaining: 32.4s
520:	learn: 0.1486658	total: 2m 9s	remaining: 32.1s
521:	learn: 0.1486619	total: 2m 10s	remaining: 31.9s
522:	learn: 0.1486561	total: 2m 10s	remaining: 31.6s
523:	learn: 0.1486518	total: 2m 10s	remaining: 31.4s
524:	learn: 0.1486441	total: 2m 10s	remaining: 31.1s
525:	learn: 0.1486375	total: 2m 11s	remaining: 30.9s
526:	learn: 0.1486308	total: 2m 11s	remaining: 30.6s
527:	learn: 0.1486223	total: 2m 11s	remaining: 30.4s
528:	learn: 0.1486157	total: 2m 11s	remaining: 30.1s
529:	learn: 0.1486113	total: 2m 12s	remaining: 29.9s
530:	learn: 0.1486105	total: 2m 12s	remaining: 29.6s
531:	learn: 0.1486042	total: 2m 12s	remaining: 29.4s
532:	learn: 0.1485987	total: 2m 12s	remaining: 29.1s
533:	learn: 0.1485935	total: 2m 13s	remaining: 28.9s
534:	learn: 0.1485924	total: 2m 13s	remaining: 28.6s
535:	learn: 0.1485829	total: 2m 13s	remaining: 28.4s
536:	learn: 0.1485763	total: 2m 13s	remaining: 28

In [ ]:
# Save validation predictions for stacking/ensembling
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('cat_valid.csv', float_format='%.6f', index=False)

In [ ]:
# Create submission file
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('cat_submit.csv', float_format='%.6f', index=False)